<a href="https://colab.research.google.com/github/gabrielbchacon/colab_projects/blob/main/HEATMAP_HEX_VIEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando as bibliotecas





In [ ]:
!pip install folium
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install geopy
!pip install plotly_express
!pip install ipython-autotime

Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [ ]:
!pip install tqdm==4.36.0

In [ ]:
!pip install googlemaps

In [ ]:
!pip install h3

In [ ]:
!pip install geojson

In [ ]:
#importando as bibliotecas
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math

import folium
from folium import plugins
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from folium import Map, Marker, GeoJson


%matplotlib inline

%load_ext autotime

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook



time: 903 ms (started: 2021-07-20 00:42:15 +00:00)


In [ ]:
from h3 import h3

def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

import googlemaps

time: 46.7 ms (started: 2021-07-20 00:42:15 +00:00)


In [ ]:
def counts_by_hexagon(df, resolution):
    """
    Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
    Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    
    Ex counts_by_hexagon(data, 9)
    """
    df = df.iloc[:,[1,2]]
    
    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row.lat, row.lng, resolution), axis = 1)
    
    df_aggreg = df.groupby(by = "hex_id").size().reset_index()
    df_aggreg.columns = ["hex_id", "value"]
    
    df_aggreg["geometry"] =  df_aggreg.hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(x ,geo_json=True)]
                                                            }
                                                        )
    
    return df_aggreg

time: 11.2 ms (started: 2021-07-20 00:42:15 +00:00)


In [ ]:
import geojson
import json
from geojson.feature import *


def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    """
    Produce the GeoJSON for a dataframe that has a geometry column in geojson 
    format already, along with the columns hex_id and value
    
    Ex counts_by_hexagon(data)
    """    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"value" : row["value"]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result


time: 19.1 ms (started: 2021-07-20 00:42:15 +00:00)


In [ ]:
import branca.colormap as cm


def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
                   kind = "linear"):
    
    """
    Creates choropleth maps given the aggregated data.
    """    
    #colormap
    min_value = df_aggreg["value"].min()
    max_value = df_aggreg["value"].max()
    m = round ((min_value + max_value ) / 2 , 0)
    
    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'hex_id'])
    
    if initial_map is None:
        initial_map = Map(location= [-23.6835,-46.7674], zoom_start=12, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>')
        

    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)
   

    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)
    
    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind
        
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)


    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)
    
    
    
    return initial_map

time: 40.4 ms (started: 2021-07-20 01:11:31 +00:00)


In [ ]:
H3_res = 5
def geo_to_h3(row):
  return h3.geo_to_h3(lat=row.lat,lng=row.lng,resolution = H3_res)

time: 1.42 ms (started: 2021-07-20 01:10:16 +00:00)


In [ ]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
rgeocode

time: 8.57 ms (started: 2021-07-20 01:10:18 +00:00)


In [ ]:
def get_map_dest(df):
  df_dest = df[['NUM_BO', 'LONGITUDE', 'LATITUDE']].rename(columns = {'LONGITUDE': 'lng', 'LATITUDE': 'lat'}, inplace = False)
  df_dest['h3_cell'] = df_dest.apply(geo_to_h3,axis=1)
  df_h3_matrix = df_dest[['NUM_BO','h3_cell']]
  df_h3 = (df_h3_matrix
                          .groupby('h3_cell')
                          .NUM_BO
                          .agg(list)
                          .to_frame("ids")
                          .reset_index())
  df_h3['count'] =(df_h3['ids']
                      .apply(lambda NUM_BO:len(NUM_BO)))
  # Counts how many points are within the hex
  df_dest_aggreg = counts_by_hexagon(df = df_dest, resolution = 9)
  df_dest_aggreg.sort_values(by = "value", ascending = False, inplace = True)
  # Creates a map using Folium
  hexmap = choropleth_map(df_aggreg = df_dest_aggreg, with_legend = True)

  df_dest_aggreg_9 = counts_by_hexagon(df = df_dest, resolution = 9)
  df_dest_aggreg_8 = counts_by_hexagon(df = df_dest, resolution = 8)
  df_dest_aggreg_7 = counts_by_hexagon(df = df_dest, resolution = 7)
  df_dest_aggreg_6 = counts_by_hexagon(df = df_dest, resolution =6)
  df_dest_aggreg_5 = counts_by_hexagon(df = df_dest, resolution =5)

  hexmap9_dest = choropleth_map(df_aggreg = df_dest_aggreg_9,with_legend = False)
  hexmap8_dest = choropleth_map(df_aggreg = df_dest_aggreg_8, initial_map = hexmap9_dest, with_legend = True)
  hexmap7_dest = choropleth_map(df_aggreg = df_dest_aggreg_7, initial_map = hexmap8_dest , with_legend = False)
  hexmap6_dest = choropleth_map(df_aggreg = df_dest_aggreg_6, initial_map = hexmap7_dest, with_legend = False)
  hexmap5_dest = choropleth_map(df_aggreg = df_dest_aggreg_5, initial_map = hexmap6_dest, with_legend = False)


  folium.map.LayerControl('bottomright', collapsed=False).add_to(hexmap6_dest)
  return hexmap9_dest 

time: 27.8 ms (started: 2021-07-20 01:53:37 +00:00)


# Aplicação

In [ ]:
df = pd.read_csv('/content/crimes (1).csv', encoding = "ISO-8859-1")
df



,Unnamed: 0,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
0,0,2018,8741,8741/2018,31/12/2018 23:50:15,01/01/2019 00:06:43,31/12/2018,23:30,A NOITE,31/12/2018,31/12/2018 23:50:15,Desconhecida,NÃ£o,NaN,AVENIDA MARECHAL TITO,3000.0,VILA CURUCA,S.PAULO,SP,-23.493959,-46.417180,Via pÃºblica,NaN,BO PARA REGISTRO,04Âº D.P. GUARULHOS,59Âº D.P. JARDIM DOS IPES,TÃ­tulo II - PatrimÃ´nio (arts. 155 a 183),Roubo (art. 157) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FJT8505,SP,SAO PAULO,Preta,CHEVROLET/ONIX 1.0MT LT,2013.0,2013.0,AUTOMOVEL,1.0,Samsung
1,1,2018,5767,5767/2018,31/12/2018 23:52:58,01/01/2019 00:18:30,31/12/2018,23:20,A NOITE,31/12/2018,31/12/2018 23:52:58,Desconhecida,NÃ£o,NaN,RUA HIPOLITO DE CAMARGO,100.0,GUAIANASES,S.PAULO,SP,-23.543206,-46.415893,Via pÃºblica,NaN,BO PARA INVESTIGAÃÃO,44Âº D.P. GUAIANAZES,44Âº D.P. GUAIANAZES,TÃ­tulo II - PatrimÃ´nio (arts. 155 a 183),Roubo (art. 157) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOX4452,SP,SAO PAULO,Vermelho,VW/FOX 1.0,2007.0,2008.0,AUTOMOVEL,1.0,Motorola
2,2,2018,4244,4244/2018,31/12/2018 23:57:19,01/01/2019 00:24:54,31/12/2018,NaN,A NOITE,31/12/2018,31/12/2018 23:57:19,Desconhecida,NÃ£o,NaN,AVENIDA JOSE PINHEIRO BORGES,2700.0,ITAQUERA,S.PAULO,SP,-23.526569,-46.432396,Via pÃºblica,NaN,ENCAMINHAMENTO DP ÃREA DO FATO,103Âº D.P. COHABII/ITAQU.,68Âº D.P. LAGEADO,TÃ­tulo II - PatrimÃ´nio (arts. 155 a 183),Roubo (art. 157) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PUN2058,SP,SAO PAULO,Cinza,FIAT/UNO VIVACE 1.0,2014.0,2014.0,AUTOMOVEL,1.0,Xiaomi
3,3,2018,12901,12901/2018,31/12/2018 21:26:42,01/01/2019 00:26:44,31/12/2018,19:10,A NOITE,31/12/2018,31/12/2018 21:26:42,Conhecida,NÃ£o,NaN,RUA SÃO LAZARO DE JERUSALEM,10.0,LAJEADO,S.PAULO,SP,-23.521743,-46.418703,Via pÃºblica,IC,ENCAMINHAMENTO DP ÃREA DO FATO,50Âº D.P. ITAIM PAULISTA,67Âº D.P. JARDIM ROBRU,LocalizaÃ§Ã£o e/ou DevoluÃ§Ã£o,LocalizaÃ§Ã£o/ApreensÃ£o e Entrega de veÃ­culo,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,LG
4,4,2018,12901,12901/2018,31/12/2018 21:26:42,01/01/2019 00:26:44,31/12/2018,19:10,A NOITE,31/12/2018,31/12/2018 21:26:42,Conhecida,NÃ£o,NaN,RUA SÃO LAZARO DE JERUSALEM,10.0,LAJEADO,S.PAULO,SP,-23.521743,-46.418703,Via pÃºblica,IC,ENCAMINHAMENTO DP ÃREA DO FATO,50Âº D.P. ITAIM PAULISTA,67Âº D.P. JARDIM ROBRU,LocalizaÃ§Ã£o e/ou DevoluÃ§Ã£o,LocalizaÃ§Ã£o/ApreensÃ£o e Entrega de veÃ­culo,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DUM3446,SP,SAO PAULO,Cinza,GM/CORSA HATCH MAXX,2007.0,2007.0,AUTOMOVEL,1.0,LG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74336,7796,2021,2284,2284/2021,30/04/2021 23:52:32,30/04/2021 23:52:32,30/04/2021,16:40,A TARDE,01/05/2021,30/04/2021 23:52:32,Conhecida,Sim,NaN,PARQUE DOM PEDRO II,1.0,SE,S.PAULO,SP,-23.545025,-46.631091,Via pÃºblica,NaN,BO PARA FLAGRANTE,08Âº D.P. BRAS,01Âº D.P. SE,TÃ­tulo II - PatrimÃ´nio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
74337,7797,2021

time: 1.04 s (started: 2021-07-20 01:42:38 +00:00)


In [ ]:
df['ESPECIE'].value_counts()

TÃ­tulo II - PatrimÃ´nio (arts. 155 a 183)                   68442
LocalizaÃ§Ã£o e/ou DevoluÃ§Ã£o                                3978
Ato infracional                                                785
TÃ­tulo I - Pessoa (arts. 121 a 154)                           269
Outros - nÃ£o criminal                                         149
L 8069/90 - ECA                                                123
TÃ­tulo XI - AdministraÃ§Ã£o pÃºblica (arts. 312 a 359-H)      115
Acidente de trÃ¢nsito                                          108
TÃ­tulo IX - Paz pÃºblica (arts. 286 a 288)                     69
Captura procurado                                               59
L 10826/03 - Estatuto do Desarmamento                           58
ApreensÃ£o de Adolescente                                       43
L 9503/97 - CÃ³digo de TrÃ¢nsito Brasileiro                     39
TÃ­tulo X - FÃ© pÃºblica (arts. 289 a 311)                      34
Pessoa                                                        

time: 18.6 ms (started: 2021-07-20 02:08:48 +00:00)


In [ ]:
df[df['RUBRICA'] == 'Tortura (Art. 1Âº)']

,Unnamed: 0,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
28705,8675,2019,2240,2240/2019,21/03/2019 07:44:50,21/03/2019 18:17:16,21/03/2019,03:30,DE MADRUGADA,21/03/2019,21/03/2019 07:44:50,Conhecida,Sim,NaN,AVENIDA AIRTON PRETINI,1.0,TATUAPE,S.PAULO,SP,-23.523772,-46.557853,Via pÃºblica,NaN,BO PARA FLAGRANTE,10Âº D.P. PENHA DE FRANCA,10Âº D.P. PENHA DE FRANCA,L 9455/97 - Tortura,Tortura (Art. 1Âº),NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2.0,Apple


time: 87.3 ms (started: 2021-07-20 02:07:05 +00:00)


In [ ]:
df_2019 = df[df['ANO_BO'] == 2019]
df_2021 = df[df['ANO_BO'] == 2021]

time: 63.5 ms (started: 2021-07-20 01:07:28 +00:00)


In [ ]:
get_map_dest(df_2021[df_2021['BAIRRO'] == 'ALTO DE PINHEIROS'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



time: 111 ms (started: 2021-07-20 01:53:42 +00:00)


In [ ]:
get_map_dest(df_2019[df_2019['BAIRRO'] == 'ALTO DE PINHEIROS'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



time: 126 ms (started: 2021-07-20 01:28:24 +00:00)


In [ ]:
df_2021_ALTOP = df_2021[df_2021['BAIRRO'] == 'ALTO DE PINHEIROS']
df_2019_ALTOP = df_2019[df_2019['BAIRRO'] == 'ALTO DE PINHEIROS']

time: 15.4 ms (started: 2021-07-20 01:39:01 +00:00)


In [ ]:
df_2019_ALTOP['RUBRICA'].value_counts()

Roubo (art. 157) - TRANSEUNTE                      51
Roubo (art. 157) - OUTROS                          33
Roubo (art. 157) - VEICULO                         18
Roubo (art. 157) - INTERIOR DE VEICULO             13
Roubo (art. 157) - CARGA                            6
Outros nÃ£o criminal                                4
Encontro de Pessoa                                  2
Roubo (art. 157) - RESIDENCIA                       1
Roubo (art. 157) - ESTABELECIMENTO COMERCIAL        1
Roubo (art. 157) - INTERIOR TRANSPORTE COLETIVO     1
Captura de procurado                                1
ExtorsÃ£o (art. 158) - PESSOA                       1
Roubo (art. 157) - INTERIOR ESTABELECIMENTO         1
LocalizaÃ§Ã£o/ApreensÃ£o e Entrega de veÃ­culo      1
Name: RUBRICA, dtype: int64

time: 9.13 ms (started: 2021-07-20 01:40:16 +00:00)


In [ ]:
df_2021_ALTOP['RUBRICA'].value_counts()

Roubo (art. 157) - OUTROS                             35
Roubo (art. 157) - TRANSEUNTE                         26
Roubo (art. 157) - VEICULO                             7
LocalizaÃ§Ã£o/ApreensÃ£o de veÃ­culo                   6
LocalizaÃ§Ã£o/ApreensÃ£o e Entrega de objeto           6
ReceptaÃ§Ã£o (art. 180) - VEICULO                      6
Roubo (art. 157) - JOALHERIA                           6
Roubo (art. 157) - INTERIOR DE VEICULO                 5
LocalizaÃ§Ã£o/ApreensÃ£o e Entrega de veÃ­culo         2
Roubo (art. 157) - Saidinha de Banco                   1
Roubo (art. 157) - APLICATIVO DE MOBILIDADE URBANA     1
Roubo (art. 157) - VEÃCULO COMPARTILHADO              1
Roubo (art. 157) - INTERIOR ESTABELECIMENTO            1
Name: RUBRICA, dtype: int64

time: 11.8 ms (started: 2021-07-20 01:40:32 +00:00)
